# 2D Puzzle Solver

This notebook solves a 2D grid puzzle where we need to:
1. Find pairs of number cells whose sum equals a target sum
2. Connect these pairs with non-overlapping paths

## Problem Constraints:
- Grid is n x n
- Some cells contain positive numbers (m cells, m is even)
- Paths can only move: left, right, up, down
- Paths cannot go through number cells (except start/end)
- Two paths cannot share the same empty cell

In [9]:
from typing import List, Tuple, Set, Optional
from collections import deque

def solve_2d_puzzle(n: int, number_cells: List[Tuple[int, int, int]], target_sum: int) -> List[List[Tuple[int, int]]]:
    """
    Solve the 2D puzzle by finding pairs of number cells that sum to target_sum
    and connecting them with non-overlapping paths.
    
    Args:
        n: Grid size (n x n)
        number_cells: List of tuples (x, y, value) for cells with numbers
        target_sum: Target sum for pairs
        
    Returns:
        List of paths, where each path is a list of (x, y) coordinates.
        Returns empty list if no solution exists.
    """
    # Create a grid to mark number cells
    grid = [[0 for _ in range(n)] for _ in range(n)]
    number_positions = {}  # (x, y) -> value
    
    for x, y, value in number_cells:
        grid[x][y] = value
        number_positions[(x, y)] = value
    
    # Find all valid pairs that sum to target_sum
    valid_pairs = []
    cells_list = list(number_positions.items())
    
    for i in range(len(cells_list)):
        for j in range(i + 1, len(cells_list)):
            (x1, y1), val1 = cells_list[i]
            (x2, y2), val2 = cells_list[j]
            if val1 + val2 == target_sum:
                valid_pairs.append(((x1, y1), (x2, y2)))
    
    if not valid_pairs:
        return []
    
    # Try to find a valid combination of pairs and paths
    # We need to use all number cells exactly once
    return find_valid_solution(n, grid, number_positions, valid_pairs, target_sum)

In [13]:
def find_path_bfs(n: int, grid: List[List[int]], start: Tuple[int, int], 
                  end: Tuple[int, int], used_cells: Set[Tuple[int, int]]) -> Optional[List[Tuple[int, int]]]:
    """
    Find a path from start to end using BFS, avoiding used cells and number cells.
    
    Args:
        n: Grid size
        grid: Grid with number cells marked
        start: Starting position (x, y)
        end: Ending position (x, y)
        used_cells: Set of cells already used by other paths
        
    Returns:
        Path as list of (x, y) coordinates, or None if no path exists
    """
    if start == end:
        return [start]
    
    queue = deque([(start, [start])])
    visited = {start}
    
    directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]  # right, left, down, up
    
    while queue:
        (x, y), path = queue.popleft()
        
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            
            # Check bounds
            if nx < 0 or nx >= n or ny < 0 or ny >= n:
                continue
            
            # Check if it's the destination
            if (nx, ny) == end:
                return path + [(nx, ny)]
            
            # Skip if already visited in this search
            if (nx, ny) in visited:
                continue
            
            # Skip if used by another path
            if (nx, ny) in used_cells:
                continue
            
            # Skip if it's a number cell (but not start or end)
            if grid[nx][ny] != 0 and (nx, ny) != start and (nx, ny) != end:
                continue
            
            visited.add((nx, ny))
            queue.append(((nx, ny), path + [(nx, ny)]))
    
    return None

In [15]:
def find_valid_solution(n: int, grid: List[List[int]], number_positions: dict,
                        valid_pairs: List[Tuple[Tuple[int, int], Tuple[int, int]]],
                        target_sum: int) -> List[List[Tuple[int, int]]]:
    """
    Find a valid solution by trying different combinations of pairs.
    Uses backtracking to find non-overlapping paths.
    """
    # We need to use all number cells exactly once
    all_number_cells = set(number_positions.keys())
    
    # Try to find a matching that uses all cells
    # Use backtracking to find valid path combinations
    def backtrack(pairs_to_use: List[Tuple[Tuple[int, int], Tuple[int, int]]],
                  used_cells: Set[Tuple[int, int]],
                  paths: List[List[Tuple[int, int]]]) -> Optional[List[List[Tuple[int, int]]]]:
        # Check if we've used all number cells
        used_number_cells = set()
        for path in paths:
            used_number_cells.add(path[0])
            used_number_cells.add(path[-1])
        
        if len(used_number_cells) == len(all_number_cells):
            return paths
        
        # Try each remaining pair
        for pair in pairs_to_use:
            start, end = pair
            
            # Skip if either cell is already used
            if start in used_number_cells or end in used_number_cells:
                continue
            
            # Try to find a path
            path = find_path_bfs(n, grid, start, end, used_cells)
            
            if path is not None:
                # Add path cells (excluding start and end) to used_cells
                new_used_cells = used_cells.copy()
                for cell in path[1:-1]:  # Exclude start and end
                    new_used_cells.add(cell)
                
                # Recursively try to complete the solution
                remaining_pairs = [p for p in pairs_to_use if p != pair]
                result = backtrack(remaining_pairs, new_used_cells, paths + [path])
                
                if result is not None:
                    return result
        
        return None
    
    # Try all possible ways to partition number cells into pairs
    # First, find all possible perfect matchings
    all_cells = list(all_number_cells)
    
    def generate_matchings(cells: List[Tuple[int, int]], 
                          current_matching: List[Tuple[Tuple[int, int], Tuple[int, int]]],
                          matchings: List[List[Tuple[Tuple[int, int], Tuple[int, int]]]]):
        if not cells:
            matchings.append(current_matching[:])
            return
        
        if len(cells) < 2:
            return
        
        first = cells[0]
        for i in range(1, len(cells)):
            second = cells[i]
            # Check if this pair is valid (sums to target)
            if (first, second) in valid_pairs or (second, first) in valid_pairs:
                pair = (first, second) if (first, second) in valid_pairs else (second, first)
                remaining = cells[1:i] + cells[i+1:]
                current_matching.append(pair)
                generate_matchings(remaining, current_matching, matchings)
                current_matching.pop()
    
    all_matchings = []
    generate_matchings(all_cells, [], all_matchings)
    
    # Try each matching to find valid paths
    for matching in all_matchings:
        result = backtrack(matching, set(), [])
        if result is not None:
            return result
    
    return []

## Test Cases

Let's test the solver with some examples:

In [16]:
# Visualization function
def visualize_solution(n: int, number_cells: List[Tuple[int, int, int]], 
                       paths: List[List[Tuple[int, int]]]):
    """Visualize the grid and paths"""
    grid = [['.' for _ in range(n)] for _ in range(n)]
    
    # Mark number cells
    for x, y, value in number_cells:
        grid[x][y] = str(value)
    
    # Mark paths
    for path_idx, path in enumerate(paths):
        for i, (x, y) in enumerate(path):
            if grid[x][y] == '.':
                # Use different symbols for different paths
                symbols = ['#', '*', '+', '=', '-', '~']
                grid[x][y] = symbols[path_idx % len(symbols)]
            elif i == 0 or i == len(path) - 1:
                # Start/end cells keep their numbers
                pass
    
    print("\nGrid visualization:")
    for row in grid:
        print(' '.join(f'{cell:>3}' for cell in row))


In [18]:
# Test Case 1: Simple 3x3 grid
n1 = 3
number_cells1 = [
    (0, 0, 5),  # Top-left: 5
    (2, 2, 3),  # Bottom-right: 3
]
target_sum1 = 8

result1 = solve_2d_puzzle(n1, number_cells1, target_sum1)
print("Test Case 1:")
print(f"Grid size: {n1}x{n1}")
print(f"Number cells: {number_cells1}")
print(f"Target sum: {target_sum1}")
print(f"Result: {result1}")
if result1:
    for i, path in enumerate(result1):
        print(f"  Path {i+1}: {path}")
    # Visualize the solution
    visualize_solution(n1, number_cells1, result1)
else:
    print("  No solution found")

Test Case 1:
Grid size: 3x3
Number cells: [(0, 0, 5), (2, 2, 3)]
Target sum: 8
Result: [[(0, 0), (0, 1), (0, 2), (1, 2), (2, 2)]]
  Path 1: [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2)]

Grid visualization:
  5   #   #
  .   .   #
  .   .   3


In [19]:
# Test Case 2: 4x4 grid with multiple pairs (designed to have non-overlapping paths)
n2 = 4
number_cells2 = [
    (0, 0, 2),  # Top-left
    (3, 0, 5),  # Bottom-left (2+5=7)
    (0, 3, 4),  # Top-right
    (3, 3, 3),  # Bottom-right (4+3=7)
]
target_sum2 = 7  # 2+5=7, 4+3=7

result2 = solve_2d_puzzle(n2, number_cells2, target_sum2)
print("\nTest Case 2:")
print(f"Grid size: {n2}x{n2}")
print(f"Number cells: {number_cells2}")
print(f"Target sum: {target_sum2}")
print(f"Result: {result2}")
if result2:
    for i, path in enumerate(result2):
        print(f"  Path {i+1}: {path}")
    # Visualize the solution
    visualize_solution(n2, number_cells2, result2)
else:
    print("  No solution found")


Test Case 2:
Grid size: 4x4
Number cells: [(0, 0, 2), (3, 0, 5), (0, 3, 4), (3, 3, 3)]
Target sum: 7
Result: [[(0, 3), (1, 3), (2, 3), (3, 3)], [(0, 0), (1, 0), (2, 0), (3, 0)]]
  Path 1: [(0, 3), (1, 3), (2, 3), (3, 3)]
  Path 2: [(0, 0), (1, 0), (2, 0), (3, 0)]

Grid visualization:
  2   .   .   4
  *   .   .   #
  *   .   .   #
  5   .   .   3


In [20]:
# Test Case 3: More complex example
n3 = 5
number_cells3 = [
    (0, 0, 1),
    (0, 4, 2),
    (4, 0, 3),
    (4, 4, 4),
]
target_sum3 = 5  # 1+4=5, 2+3=5

result3 = solve_2d_puzzle(n3, number_cells3, target_sum3)
print("\nTest Case 3:")
print(f"Grid size: {n3}x{n3}")
print(f"Number cells: {number_cells3}")
print(f"Target sum: {target_sum3}")
print(f"Result: {result3}")
if result3:
    for i, path in enumerate(result3):
        print(f"  Path {i+1} (length {len(path)}): {path}")
    visualize_solution(n3, number_cells3, result3)
else:
    print("  No solution found")


Test Case 3:
Grid size: 5x5
Number cells: [(0, 0, 1), (0, 4, 2), (4, 0, 3), (4, 4, 4)]
Target sum: 5
Result: []
  No solution found


In [21]:
# Test Case 4: No solution case (pairs exist but paths overlap)
n4 = 3
number_cells4 = [
    (0, 0, 1),
    (0, 2, 2),
    (2, 0, 3),
    (2, 2, 4),
]
target_sum4 = 5  # 1+4=5, 2+3=5

result4 = solve_2d_puzzle(n4, number_cells4, target_sum4)
print("\nTest Case 4:")
print(f"Grid size: {n4}x{n4}")
print(f"Number cells: {number_cells4}")
print(f"Target sum: {target_sum4}")
print(f"Result: {result4}")
if result4:
    for i, path in enumerate(result4):
        print(f"  Path {i+1}: {path}")
    visualize_solution(n4, number_cells4, result4)
else:
    print("  No solution found (as expected if paths would overlap)")


Test Case 4:
Grid size: 3x3
Number cells: [(0, 0, 1), (0, 2, 2), (2, 0, 3), (2, 2, 4)]
Target sum: 5
Result: []
  No solution found (as expected if paths would overlap)


In [23]:
# Test Case 5: More complex example
n5 = 9
number_cells5 = [
    (0, 0, 2.3),
    (1, 7, 2.8),
    (3, 4, 1.3),
    (3, 6, 1.2),
    (4, 0, 3.8),
    (7, 1, 2.2),
    (8, 5, 3.7),
    (8, 8, 2.7),
]
target_sum5 = 5  # 1+4=5, 2+3=5

result5 = solve_2d_puzzle(n5, number_cells5, target_sum3)
print("\nTest Case 5:")
print(f"Grid size: {n5}x{n5}")
print(f"Number cells: {number_cells5}")
print(f"Target sum: {target_sum5}")
print(f"Result: {result5}")
if result5:
    for i, path in enumerate(result5):
        print(f"  Path {i+1} (length {len(path)}): {path}")
    visualize_solution(n5, number_cells5, result5)
else:
    print("  No solution found")


Test Case 5:
Grid size: 9x9
Number cells: [(0, 0, 2.3), (1, 7, 2.8), (3, 4, 1.3), (3, 6, 1.2), (4, 0, 3.8), (7, 1, 2.2), (8, 5, 3.7), (8, 8, 2.7)]
Target sum: 5
Result: [[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (1, 8), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (7, 8), (8, 8)], [(1, 7), (1, 6), (1, 5), (1, 4), (1, 3), (1, 2), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)], [(3, 6), (3, 5), (4, 5), (4, 4), (4, 3), (4, 2), (5, 2), (6, 2), (7, 2), (8, 2), (8, 1), (8, 0), (7, 0), (6, 0), (5, 0), (4, 0)], [(3, 4), (2, 4), (2, 5), (2, 6), (2, 7), (3, 7), (4, 7), (4, 6), (5, 6), (5, 5), (6, 5), (7, 5), (8, 5)]]
  Path 1 (length 17): [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (1, 8), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (7, 8), (8, 8)]
  Path 2 (length 13): [(1, 7), (1, 6), (1, 5), (1, 4), (1, 3), (1, 2), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
  Path 3 (length 16): [(3, 6), (3, 5), (4, 5), (4, 4), (4, 3), 

In [24]:
# Test Case 6: More complex example
n6 = 9
number_cells6 = [
    (0, 0, 2.3),
    (1, 7, 2.8),
    (2, 1, 4.8),
    (3, 4, 1.3),
    (3, 6, 1.2),
    (4, 0, 3.8),
    (7, 1, 2.2),
    (7, 6, 0.2),
    (8, 5, 3.7),
    (8, 8, 2.7),
]
target_sum6 = 5  # 1+4=5, 2+3=5

result6 = solve_2d_puzzle(n6, number_cells6, target_sum6)
print("\nTest Case 6:")
print(f"Grid size: {n6}x{n6}")
print(f"Number cells: {number_cells6}")
print(f"Target sum: {target_sum6}")
print(f"Result: {result6}")
if result6:
    for i, path in enumerate(result6):
        print(f"  Path {i+1} (length {len(path)}): {path}")
    visualize_solution(n6, number_cells6, result6)
else:
    print("  No solution found")


Test Case 6:
Grid size: 9x9
Number cells: [(0, 0, 2.3), (1, 7, 2.8), (2, 1, 4.8), (3, 4, 1.3), (3, 6, 1.2), (4, 0, 3.8), (7, 1, 2.2), (7, 6, 0.2), (8, 5, 3.7), (8, 8, 2.7)]
Target sum: 5
Result: []
  No solution found


## Algorithm Explanation

The solution uses a backtracking approach:

1. **Find Valid Pairs**: Identify all pairs of number cells whose sum equals the target sum
2. **Generate Matchings**: Find all possible ways to partition number cells into pairs (perfect matching)
3. **Path Finding**: For each matching, use BFS to find paths between pairs
4. **Conflict Resolution**: Ensure paths don't share empty cells by tracking used cells
5. **Backtracking**: If a path combination fails, backtrack and try another matching

The algorithm ensures:
- All number cells are used exactly once
- Paths only use empty cells (except start/end)
- No two paths share the same empty cell
- Paths only move in 4 directions (up, down, left, right)